In [ ]:
# Install dependencies
import subprocess
import sys
import importlib.util

dependencies = ['pandas', 'numpy', 'matplotlib', 'seaborn', 'prophet', 'ipywidgets', 'plotly']
for pkg in dependencies:
    if importlib.util.find_spec(pkg) is None:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", pkg, "--user", "--quiet"])
        except subprocess.CalledProcessError as e:
            print(f"❌ Failed to install {pkg}: {e}")
            print(f"Run in terminal: {sys.executable} -m pip install {pkg} --user --verbose")
            raise
print("✅ Dependencies installed.")

In [ ]:
import pandas as pd
import numpy as np
import os

# Create sample data if the CSV file doesn't exist
os.makedirs('data', exist_ok=True)
if not os.path.exists('data/marketing_trends.csv'):
    try:
        dates = pd.date_range(start='2020-01-01', end='2023-12-31', freq='ME')
        data = {
            'month': dates,
            'spend': np.random.uniform(1000, 10000, len(dates)),
            'conversions': np.random.randint(50, 500, len(dates))
        }
        df = pd.DataFrame(data)
        df.to_csv('data/marketing_trends.csv', index=False)
        print("✅ Sample data created and saved to 'data/marketing_trends.csv'")
    except Exception as e:
        print(f"❌ Error creating sample data: {e}")
        raise

# Load data
try:
    df = pd.read_csv('data/marketing_trends.csv')
    print(f"✅ Data Loaded. Total Records: {len(df)}")
    display(df.head())
except FileNotFoundError:
    print("❌ Error: 'data/marketing_trends.csv' not found.")
    raise
except Exception as e:
    print(f"❌ Error loading data: {e}")
    raise

In [ ]:
if 'df' in globals():
    try:
        df['month'] = pd.to_datetime(df['month'])
        print("📊 Dataset Info:")
        print(df.info())
        print("\n📊 Spend & Conversions Summary:")
        display(df.describe())
    except Exception as e:
        print(f"❌ Error processing data: {e}")
        raise
else:
    print("❌ Error: DataFrame 'df' not defined.")
    raise

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

if 'df' in globals() and all(col in df.columns for col in ['month', 'spend', 'conversions']):
    try:
        plt.figure(figsize=(10,5))
        sns.lineplot(data=df, x='month', y='spend', label='Spend')
        sns.lineplot(data=df, x='month', y='conversions', label='Conversions')
        plt.title("Marketing Spend & Conversions Over Time")
        plt.legend()
        plt.show()
    except Exception as e:
        print(f"❌ Error plotting data: {e}")
        raise
else:
    print("⚠️ Required columns ('month', 'spend', 'conversions') missing.")
    raise

In [ ]:
if 'df' in globals() and all(col in df.columns for col in ['month', 'conversions']):
    try:
        forecast_df = df[['month', 'conversions']].rename(columns={'month': 'ds', 'conversions': 'y'})
        display(forecast_df.head())
    except Exception as e:
        print(f"❌ Error preparing forecast data: {e}")
        raise
else:
    print("⚠️ Required columns ('month', 'conversions') missing.")
    raise

In [ ]:
import logging

if 'forecast_df' in globals():
    try:
        from prophet import Prophet
        # Suppress cmdstanpy info logs
        logging.getLogger('cmdstanpy').setLevel(logging.WARNING)
        model = Prophet()
        model.fit(forecast_df)
        print("✅ Prophet model trained.")
    except Exception as e:
        print(f"❌ Error training Prophet model: {e}")
        raise
else:
    print("❌ Error: DataFrame 'forecast_df' not defined.")
    raise

In [ ]:
import logging

if 'model' in globals():
    try:
        # Suppress cmdstanpy info logs
        logging.getLogger('cmdstanpy').setLevel(logging.WARNING)
        future = model.make_future_dataframe(periods=12, freq='ME')
        forecast = model.predict(future)
        display(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail())
    except Exception as e:
        print(f"❌ Error making forecast: {e}")
        raise
else:
    print("❌ Error: 'model' not defined.")
    raise

In [ ]:
if 'model' in globals() and 'forecast' in globals():
    try:
        fig1 = model.plot(forecast)
        plt.title("Forecasted Conversions")
        plt.show()
    except Exception as e:
        print(f"❌ Error plotting forecast: {e}")
        raise
else:
    print("❌ Error: 'model' or 'forecast' not defined.")
    raise

In [ ]:
if 'forecast' in globals():
    try:
        forecast.to_csv('data/marketing_conversions_forecast.csv', index=False)
        print("✅ Forecast saved to 'data/marketing_conversions_forecast.csv'")
    except Exception as e:
        print(f"❌ Error saving file: {e}")
        raise
else:
    print("❌ Error: 'forecast' not defined.")
    raise